# 단어 수준의 번역기

In [185]:
import numpy as np
import pandas as pd
import re
import shutil
import os
import unicodedata
import urllib3
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

In [186]:
import os
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 178009


,eng,fra,cc
88866,You're not a very good liar.,Tu n'es pas une très bonne menteuse.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
13630,All are welcomed.,Toutes sont bienvenues.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
114899,Canned food doesn't interest her.,La nourriture en conserve ne l'intéresse pas.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
113768,We managed to get there on time.,On est parvenu à s'y rendre à temps.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
160526,They shook hands when they met at the airport.,Ils se serrèrent la main lorsqu'ils se rencont...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...


In [187]:
lines = lines[['eng', 'fra']][:33000] #33000개 샘플 사용
lines.sample(5)

,eng,fra
27733,You can't leave me.,Tu ne peux pas me quitter.
21326,They'll eat those.,Elles mangeront celles-là.
32527,Tom was my cellmate.,Tom était mon compagnon de cellule.
8638,Is it reliable?,Est-ce fiable ?
21099,The bar is closed.,Le bar est fermé.


## 데이터 전처리

In [188]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

In [189]:
def preprocess_sentence(sent):
    # 위에서 구현한 함수를 내부적으로 호출
    sent = unicode_to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백을 만듭니다.
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,¿])", r" \1", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

    sent = re.sub(r"\s+", " ", sent)
    return sent

In [190]:
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"
print(preprocess_sentence(en_sent))
print(preprocess_sentence(fr_sent))

have you had dinner ?
avez vous deja dine ?


In [191]:
lines.eng = lines.eng.apply(lambda x : preprocess_sentence(x))
lines.fra = lines.fra.apply(lambda x : preprocess_sentence(x))

In [192]:
lines.sample(5)

,eng,fra
13735,attention please !,attention s il vous plait !
9143,tell everybody .,dis le a tout le monde .
25191,i ve seen too much .,j en ai trop vu .
4291,now drink up .,finissez votre verre maintenant !
2284,i like math .,j aime les maths .


In [193]:
# 시작 토큰과 종료 토큰 추가
sos_token = '<start>'
eos_token = '<end>'
lines.fra = lines.fra.apply(lambda x : '<start> '+ x + ' <end>')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 33000


,eng,fra
14680,i didn t mean it .,<start> ce n etait pas mon intention . <end>
11064,i heard a noise .,<start> j ai entendu un bruit . <end>
14284,he is very young .,<start> il est tres jeune . <end>
17468,which is cheaper ?,<start> laquelle est la moins chere ? <end>
8120,i like singing .,<start> j aime chanter . <end>


In [ ]:
def tokenize(sentence):
    tokenizer = Tokenizer(num_words=7000, filters="", oov_token="<unk>")
    tokenizer.fit_on_texts(sentence)
    tensor = tokenizer.texts_to_sequences(sentence)

In [194]:
eng_tokenizer = Tokenizer(num_words=7000, filters="", oov_token="<unk>")   # 단어 단위로 Tokenizer를 생성합니다. 
eng_tokenizer.fit_on_texts(lines.eng)               # 33000개의 행을 가진 eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(lines.eng)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

[[31, 2], [1133, 2], [1133, 2]]

In [195]:
fra_tokenizer = Tokenizer(num_words=7000, filters="", oov_token="<unk>")   # 단어 단위로 Tokenizer를 생성합니다. 
fra_tokenizer.fit_on_texts(lines.fra)                 # 33000개의 행을 가진 fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(lines.fra)     # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[2, 90, 13, 3], [2, 1021, 13, 3], [2, 1021, 4, 3]]

In [196]:
for idx in eng_tokenizer.index_word:
    print(idx, ":", eng_tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : .
3 : i
4 : you
5 : ?
6 : it
7 : a
8 : s
9 : is
10 : re


In [197]:
for idx in fra_tokenizer.index_word:
    print(idx, ":", fra_tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : .
5 : je
6 : est
7 : ?
8 : vous
9 : a
10 : pas


In [198]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 4664
프랑스어 단어장의 크기 : 8039


In [97]:
fra_tokenizer.index_word[2] = '\t'
fra_tokenizer.index_word[3] = '\n'

In [98]:
for idx in fra_tokenizer.index_word:
    print(idx, ":", fra_tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : 	
3 : 

4 : .
5 : je
6 : est
7 : ?
8 : vous
9 : a
10 : pas


In [199]:
max_eng_seq_len = max([len(line) for line in encoder_input])
max_fra_seq_len = max([len(line) for line in decoder_input])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 16


In [200]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 4664
프랑스어 단어장의 크기 : 8039
영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 16


In [201]:
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [202]:
print(encoder_input[:3])
print(decoder_input[:3])
print(decoder_target[:3])

[[31, 2], [1133, 2], [1133, 2]]
[[2, 90, 13], [2, 1021, 13], [2, 1021, 4]]
[[90, 13, 3], [1021, 13, 3], [1021, 4, 3]]


In [142]:

fra_tokenizer.index_word[1019]

'salut'

In [203]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 8)
프랑스어 입력데이터의 크기(shape) : (33000, 16)
프랑스어 출력데이터의 크기(shape) : (33000, 16)


In [144]:
print(encoder_input[0])

[30  0  0  0  0  0  0  0]


In [204]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input_train))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input_train))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target_train))

영어 학습데이터의 크기(shape) : (30000, 8)
프랑스어 학습 입력데이터의 크기(shape) : (30000, 16)
프랑스어 학습 출력데이터의 크기(shape) : (30000, 16)


In [205]:
print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(30000, 8)
(30000, 16)
(30000, 16)
(3000, 8)
(3000, 16)
(3000, 16)


## 모델 만들기

In [206]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

embedding_size = 50
hidden_size = 50

In [207]:
# 인코더에서 사용할 임베딩 층
encoder_inputs = Input(shape=(None, ))
enc_emb =  Embedding(eng_vocab_size, embedding_size,
                    input_length=max_eng_seq_len)(encoder_inputs)
enc_masking = Masking(mask_value=0.0)(enc_emb)
encoder_lstm = LSTM(hidden_size, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

In [208]:
decoder_inputs = Input(shape=(None, ), name='decoder_input')
dec_emb =  Embedding(fra_vocab_size, embedding_size)(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb)
decoder_lstm = LSTM(hidden_size, return_sequences = True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state = encoder_states)

In [209]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [210]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
decoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 50)     233200      input_4[0][0]                    
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 50)     401950      decoder_input[0][0]              
____________________________________________________________________________________________

In [211]:
model.fit(x=[encoder_input_train, decoder_input_train], 
          y=decoder_target_train, 
          validation_data = ([encoder_input_test, decoder_input_test], 
                             decoder_target_test),
          batch_size=32, 
          epochs=15)

Epoch 1/15
938/938 [==============================] - 12s 13ms/step - loss: 2.0211 - val_loss: 1.7794
Epoch 2/15
938/938 [==============================] - 10s 11ms/step - loss: 1.4029 - val_loss: 1.6122
Epoch 3/15
938/938 [==============================] - 10s 11ms/step - loss: 1.2780 - val_loss: 1.5214
Epoch 4/15
938/938 [==============================] - 11s 12ms/step - loss: 1.2004 - val_loss: 1.4588
Epoch 5/15
938/938 [==============================] - 10s 11ms/step - loss: 1.1402 - val_loss: 1.4091
Epoch 6/15
938/938 [==============================] - 10s 11ms/step - loss: 1.0914 - val_loss: 1.3612
Epoch 7/15
938/938 [==============================] - 11s 12ms/step - loss: 1.0518 - val_loss: 1.3320
Epoch 8/15
938/938 [==============================] - 11s 11ms/step - loss: 1.0211 - val_loss: 1.3090
Epoch 9/15
938/938 [==============================] - 11s 12ms/step - loss: 1.0019 - val_loss: 1.2968
Epoch 10/15
938/938 [==============================] - 11s 12ms/step - loss: 0.986

## 모델 테스트

In [227]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_3 (Embedding)      (None, None, 50)          233200    
_________________________________________________________________
masking (Masking)            (None, None, 50)          0         
_________________________________________________________________
lstm_2 (LSTM)                [(None, 50), (None, 50),  20200     
Total params: 253,400
Trainable params: 253,400
Non-trainable params: 0
_________________________________________________________________


In [228]:
# 이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape=(hidden_size,))
# 이전 time step의 cell state를 저장하는 텐서
decoder_state_input_c = Input(shape=(hidden_size,))
# 이전 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# decoder_states_inputs를 현재 time step의 초기 상태로 사용.
# 구체적인 동작 자체는 def decode_sequence()에 구현.
dec_emb2 = Embedding(fra_vocab_size, embedding_size)(decoder_inputs)
dec_masking2 = Masking(mask_value=0.0)(dec_emb2)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_masking2, initial_state = decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_softmax_layer(decoder_outputs2)

In [229]:
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs2] + decoder_states2)
decoder_model.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, None, 50)     401950      decoder_input[0][0]              
__________________________________________________________________________________________________
masking_3 (Masking)             (None, None, 50)     0           embedding_7[0][0]                
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, 50)]         0                                            
___________________________________________________________________________________________

In [230]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [240]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <start>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = fra2idx['<start>']
    
    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '<end>' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [245]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스 (자유롭게 선택해 보세요)
    input_seq = encoder_input_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.eng[seq_index])
    print('정답 문장:', lines.fra[seq_index][8:len(lines.fra[seq_index])-6]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: run !
정답 문장: cours !
번역기가 번역한 문장:  je ne . . . . . 
-----------------------------------
입력 문장: i left .
정답 문장: je suis partie .
번역기가 번역한 문장:  j maintenant de 
-----------------------------------
입력 문장: call us .
정답 문장: appelez nous !
번역기가 번역한 문장:  je me . . . . . 
-----------------------------------
입력 문장: how nice !
정답 문장: comme c est gentil !
번역기가 번역한 문장:  j me ete . . . 
-----------------------------------
입력 문장: turn left .
정답 문장: tourne a gauche .
번역기가 번역한 문장:  c fait . . . . 


## 느낀점
1. 훈련할 때와 테스트 할 때의 단계가 달라서, 모델이 새로 필요하다는 것은 이해했다 그러나 모델을 연결 시키고, 조건을 만족할 때까지 model.predict를 계속하는 방식이 생소한 것 같다.
2. 번역기 성능이 매우 나쁘다.